In [217]:
import numpy as np
import os
import pathlib
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import PIL

In [218]:
data_dir = pathlib.Path('abc/')                 
alphabats = list(data_dir.glob('*/*.jpg'))

batch_size = 64
img_height = 256
img_width = 256

In [219]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 6912 files belonging to 24 classes.
Using 4839 files for training.
Found 6912 files belonging to 24 classes.
Using 2073 files for validation.


In [220]:
class_names = train_ds.class_names
print(class_names)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']


In [221]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(64, 256, 256, 3)
(64,)


In [222]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

In [223]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image)) 

0.18431373 0.92549026


In [224]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [225]:
num_classes = 24

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [226]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [227]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_23 (Rescaling)     (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 256, 256, 16)      448       
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 128, 128, 32)      4640      
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 32, 32, 64)      

In [228]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/10
76/76 [==============================] - 294s 4s/step - loss: 2.9852 - accuracy: 0.1484 - val_loss: 2.5383 - val_accuracy: 0.2812
Epoch 2/10
76/76 [==============================] - 262s 3s/step - loss: 1.9005 - accuracy: 0.4879 - val_loss: 1.5375 - val_accuracy: 0.6112
Epoch 3/10
76/76 [==============================] - 250s 3s/step - loss: 1.0808 - accuracy: 0.7233 - val_loss: 1.3387 - val_accuracy: 0.6744
Epoch 4/10
76/76 [==============================] - 232s 3s/step - loss: 0.5951 - accuracy: 0.8483 - val_loss: 1.3098 - val_accuracy: 0.7212
Epoch 5/10
76/76 [==============================] - 253s 3s/step - loss: 0.3630 - accuracy: 0.9072 - val_loss: 1.4949 - val_accuracy: 0.7164
Epoch 6/10
76/76 [==============================] - 246s 3s/step - loss: 0.2143 - accuracy: 0.9450 - val_loss: 1.5163 - val_accuracy: 0.7405
Epoch 7/10
76/76 [==============================] - 251s 3s/step - loss: 0.1547 - accuracy: 0.9632 - val_loss: 1.6725 - val_accuracy: 0.7429
Epoch 8/10
76

In [240]:
model.evaluate(val_ds)

In [ ]:
model.save('modelweights.h5')

In [178]:
# model = tf.keras.models.load_model('modelweights.h5')

In [239]:
import cv2
img_path = "abc/H/1.jpg"
img_array = cv2.imread(img_path,1)

img_array = tf.expand_dims(img_array, 0) # Create a batch
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

This image most likely belongs to H with a 70.05 percent confidence.
